### Chatbot

In [24]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

import scipy

In [25]:
df = pd.read_excel(r"database_intents.xlsx")

In [26]:
class_names = df['Intent type'].unique()

In [27]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [28]:
df_oh = df_oh[['Intent type', 'Sentence', 'Greeting', 'Search', 'Suggestions', 'Farewell',
      'Options', 'Headers']]

In [29]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [30]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(df_train['Sentence']).batch(128)
vectorizer.adapt(text_ds)

In [31]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'entity', 'me', 'to']

In [15]:
# import pickle
# pickle.dump({'config': vectorizer.get_config(),
#              'weights': vectorizer.get_weights()}
#             , open("tv_layer.pkl", "wb"))

In [26]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [27]:
embeddings_index = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [28]:
num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 88 words (10 misses)


In [29]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [30]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          5000      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         32128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         8204

In [31]:
X_train = vectorizer(np.array([[s] for s in df_train['Sentence']])).numpy()
X_test = vectorizer(np.array([[s] for s in df_test['Sentence']])).numpy()

y_train = np.array(df_train.iloc[:, 2:])
y_test = np.array(df_test.iloc[:, 2:])

In [32]:
model.compile(
    loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(X_train, y_train, batch_size = 128, epochs = 30, validation_data=(X_test, y_test))

Epoch 1/30
1/1 [==============================] - 1s 1s/step - loss: 1.7977 - acc: 0.2169 - val_loss: 1.7351 - val_acc: 0.4444
Epoch 2/30
1/1 [==============================] - 0s 58ms/step - loss: 1.7514 - acc: 0.2892 - val_loss: 1.6247 - val_acc: 0.3611
Epoch 3/30
1/1 [==============================] - 0s 56ms/step - loss: 1.6251 - acc: 0.4337 - val_loss: 1.4749 - val_acc: 0.4167
Epoch 4/30
1/1 [==============================] - 0s 63ms/step - loss: 1.5377 - acc: 0.3976 - val_loss: 1.3287 - val_acc: 0.4444
Epoch 5/30
1/1 [==============================] - 0s 55ms/step - loss: 1.3939 - acc: 0.4578 - val_loss: 1.2191 - val_acc: 0.7500
Epoch 6/30
1/1 [==============================] - 0s 57ms/step - loss: 1.3078 - acc: 0.5060 - val_loss: 1.1333 - val_acc: 0.5278
Epoch 7/30
1/1 [==============================] - 0s 55ms/step - loss: 1.2486 - acc: 0.4337 - val_loss: 1.2613 - val_acc: 0.5000
Epoch 8/30
1/1 [==============================] - 0s 56ms/step - loss: 1.2530 - acc: 0.5060 - val_l

In [32]:
model.save('embbedings.h5')

In [3]:
model = keras.models.load_model('embbedings.h5')

In [35]:
from_disk = pickle.load(open("tv_layer.pkl", "rb"))
vectorizer = TextVectorization.from_config(from_disk['config'])
# vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorizer.set_weights(from_disk['weights'])

In [34]:
vectorizer

In [36]:
vectorizer

### Pruebas

In [37]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["I want to get information about Barcelona"]]
)

class_names[np.argmax(probabilities[0])]

TypeError: Failed to convert object of type <class 'tensorflow.python.ops.ragged.ragged_tensor.RaggedTensor'> to Tensor. Contents: tf.RaggedTensor(values=tf.RaggedTensor(values=Tensor("model_2/embedding_1/embedding_lookup_ragged/embedding_lookup/Identity_1:0", shape=(None, 50), dtype=float32), row_splits=Tensor("Placeholder_1:0", shape=(None,), dtype=int64)), row_splits=Tensor("model_2/conv1d_3/conv1d/RaggedExpandDims/RaggedFromUniformRowLength/RowPartitionFromUniformRowLength/mul:0", shape=(None,), dtype=int64)). Consider casting elements to a supported type.

In [35]:
probabilities_2 = end_to_end_model.predict(
    [["Hello, how are you?"]]
)
class_names[np.argmax(probabilities_2[0])]

'Greeting'

In [17]:
probabilities

array([[0.0118015 , 0.7195744 , 0.19317469, 0.01835446, 0.04325972,
        0.01383533]], dtype=float32)

In [36]:
probabilities_2

array([[0.9571211 , 0.00110743, 0.00148095, 0.02961333, 0.00876006,
        0.00191713]], dtype=float32)

### No lo he probado en el chatbot, para hacer pruebas usar el módulo de arriba

# CHATBOT TEST

In [83]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)
    scipy.sparse.csr_matrix.sort_indices(test_proc)
    
    gret_prob = mlp_greeting.predict(test_proc)
    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    farewell_prob = mlp_farewell.predict(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob, farewell_prob]
    print(probs)
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    elif idx == 2:
        print("Esto es una sugerencia")
    else:
        print("Esto es una despedida")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [96]:
chatbot()

hi
[array([[0.8040894]], dtype=float32), array([[0.00038779]], dtype=float32), array([[3.612887e-05]], dtype=float32), array([[0.00043699]], dtype=float32)]
Esto es un saludo


* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header

In [ ]:
chatbot()

In [ ]:
def Intent_detection_function(keyboard):
    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    farewell_prob = mlp_farewell.predict_proba(test_proc)
    cv = joblib.load("vectorizer.pkl")


    input_text = pd.DataFrame(data={'Sentence': [keyboard]})
    _, test_proc = cv.transform(input_text['Sentence'])
    #df_test_proc, test_proc = processing(test, cv=cv)

    test_proc = test_proc.toarray()

    gret_prob = mlp_greeting.predict(test_proc)

    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    fare_prob = mlp_farewell.predict(test_proc)


    probs = [gret_prob, search_prob, sugg_prob, fare_prob]
    print(probs)
    idx = np.argmax(probs)

    if idx == 0:
        intent = "Greeting"
        keyword = None
    elif idx == 1:
        intent = "Search"
        keyword = getEntities(keyboard)
    elif idx == 2:
        intent = "Suggestions"
        keyword = getEntities(keyboard)
    elif idx == 3:
        intent = "Farewell"
        keyword = None

    return intent, keyword

In [ ]:
Intent_detection_function()